In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import json
import re
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

# bring in population from the api notebook

In [ ]:
tnpop_65plus = pd.read_csv(r"C:\Users\hunte\Documents\NNS-DDA10\hunter_capstone\data\visualization_data\tnpop_65plus.csv")

# Bring in and clean TN county data for future plotting

In [ ]:
counties = gpd.read_file(r"C:\Users\hunte\Documents\NNS-DDA10\hunter_capstone\data\TIGER_Line_2018_Tennessee_Counties.geojson")

In [ ]:
counties = counties[['NAME', 'geometry']]

In [ ]:
county_areas = pd.read_csv(r"C:\Users\hunte\Documents\NNS-DDA10\hunter_capstone\data\county_areas.csv")

In [ ]:
county_areas = county_areas.rename(columns = {'Land Area ?' : 'land_area_sqmi',
                                              'County / Population' : 'county_name'})

In [ ]:
county_areas['county_name'] = county_areas['county_name'].replace('Dekalb', 'DeKalb')

In [ ]:
county_areas['county_name'] = county_areas['county_name'].replace('Mcminn', 'McMinn')

In [ ]:
county_areas['county_name'] = county_areas['county_name'].replace('Mcnairy', 'McNairy')

In [ ]:
tnpop_65plus_merge = tnpop_65plus.merge(county_areas, how = 'inner',
                                        on = 'county_name')

# bring in and clean courses data

In [ ]:
courses = gpd.read_file(r"C:\Users\hunte\Documents\NNS-DDA10\hunter_capstone\data\golf_course_locations.csv")

In [ ]:
courses['geometry'] = courses.apply(lambda x:Point((float(x.lat),
                                                    float(x.lon))),
                                    axis = 1)

In [ ]:
courses = gpd.GeoDataFrame(courses,
                           crs = counties.crs,
                           geometry = courses['geometry'])

In [ ]:
courses['state'] = courses['golf_course_name'].str[-2:]

In [ ]:
tn_courses = courses.loc[courses['state'] == 'TN']

In [ ]:
# Extract the course type from course description

In [ ]:
tn_courses['course_type'] = tn_courses['course_desc'].str.extract('\(([\w\s]+)\)')

In [ ]:
# Extract the number of holes from course description

In [ ]:
tn_courses['holes'] = tn_courses['course_desc'].str.extract('\) \(([\w\s]+)\)')

In [ ]:
# Extract the course name and city from course description

In [ ]:
tn_courses[['course_name', 'city']] = tn_courses['golf_course_name'].str.split('-', expand = True)

In [ ]:
tn_courses = tn_courses.drop(columns = {'golf_course_name', 'course_desc'})

In [ ]:
# Geospatial join to get golf courses within the counties

In [ ]:
join = gpd.sjoin(tn_courses, counties, predicate = 'within')

In [ ]:
join = join.rename(columns = {'NAME' : 'county_name'})

# Bring in the tnpop_65plus df for population and income along with county_areas

In [ ]:
county_areas = county_areas.rename(columns = {'Rank' : 'area_rank',})

In [ ]:
county_areas['land_area_sqmi'] = county_areas['land_area_sqmi'].str[:-6]

In [ ]:
county_areas_income = county_areas.merge(tnpop_65plus, how = 'inner', on = 'county_name')

In [ ]:
# Join with geospatial join to get course info including county name

In [ ]:
all_course_info = join.merge(county_areas_income, how = 'inner', on = 'county_name')

In [ ]:
all_course_info['city'] = all_course_info['city'].str[:-3]

In [ ]:
all_course_info = all_course_info[['course_name', 'course_type', 'holes', 'city', 'state', 'county_name',
                                   'median_income', 'male_pop', 'female_pop', 'pop_over_65',
                                   'total_pop', 'pct_over_65', 'land_area_sqmi', 'area_rank', 
                                   'lat', 'lon', 'geometry']]

# bring in counties regions table for further analysis

In [ ]:
counties = counties.rename(columns = {'NAME' : 'county_name'})

In [ ]:
county_region = pd.read_csv(r"C:\Users\hunte\Documents\NNS-DDA10\hunter_capstone\data\county_region.csv")

In [ ]:
# Create regions in order to do a breakdown per region

In [ ]:
east_counties = county_region['East'].reset_index(name = 'county_name').dropna()

In [ ]:
east_counties['region'] = 'East'

In [ ]:
east_counties = east_counties.drop(columns = {'index'})

In [ ]:
east_counties['state'] = 'TN'

In [ ]:
west_counties = county_region['West'].reset_index(name = 'county_name').dropna()

In [ ]:
west_counties['region'] = 'West'

In [ ]:
west_counties = west_counties.drop(columns = {'index'})

In [ ]:
west_counties['state'] = 'TN'

In [ ]:
mid_counties = county_region['Middle'].reset_index(name = 'county_name').dropna()

In [ ]:
mid_counties['region'] = 'Middle'

In [ ]:
mid_counties = mid_counties.drop(columns = {'index'})

In [ ]:
mid_counties['state'] = 'TN'

In [ ]:
county_region = pd.concat([east_counties, west_counties, mid_counties])

In [ ]:
counties = counties.merge(county_region, how = 'inner', on = 'county_name')

In [ ]:
counties = counties[['county_name', 'region', 'geometry']]

In [ ]:
# Create county geospatial dataframes in case we need it

In [ ]:
east_counties_geo = counties.loc[counties['region'] == 'East']

In [ ]:
west_counties_geo = counties.loc[counties['region'] == 'West']

In [ ]:
middle_counties_geo = counties.loc[counties['region'] == 'Middle']

In [ ]:
# Merge regions with course info to display region

In [ ]:
all_course_info = all_course_info.merge(county_region, how = 'inner', on = 'county_name')

In [ ]:
all_course_info = all_course_info[['course_name', 'course_type', 'holes', 'city', 'state_x', 'county_name',
                                   'region', 'median_income', 'male_pop', 'female_pop', 'pop_over_65',
                                   'total_pop', 'pct_over_65', 'land_area_sqmi', 'area_rank', 
                                   'lat', 'lon', 'geometry']]

In [ ]:
all_course_info = all_course_info.rename(columns = {'state_x' : 'state'})

# create dataframes for regional courses and course types

In [ ]:
east_courses = all_course_info.loc[all_course_info['region'] == 'East']

In [ ]:
west_courses = all_course_info.loc[all_course_info['region'] == 'West']

In [ ]:
mid_courses = all_course_info.loc[all_course_info['region'] == 'Middle']

In [ ]:
private_courses = all_course_info.loc[all_course_info['course_type'] == 'Private']

In [ ]:
public_courses = all_course_info.loc[all_course_info['course_type'] == 'Public']

In [ ]:
municipal_courses = all_course_info.loc[all_course_info['course_type'] == 'Municipal']

In [ ]:
semi_private_courses = all_course_info.loc[all_course_info['course_type'] == 'Semi Private']

In [ ]:
all_course_info = all_course_info.rename(columns = {'region' : 'county_region',
                                                    'median_income' : 'est_median_hh_inc',
                                                    'male_pop' : 'male_pop_over65',
                                                    'female_pop' : 'female_pop_over65'})

In [ ]:
# Create course count dataframe to merge with county_metrics to get all metrics in one dataframe

In [ ]:
course_count = all_course_info.value_counts('county_name').reset_index(name = 'course_count')

In [ ]:
county_metrics = county_areas_income.merge(course_count, how = 'left', on = 'county_name')

In [ ]:
county_metrics['course_count'] = county_metrics['course_count'].fillna(0)

In [ ]:
county_metrics['course_count'] = county_metrics['course_count'].astype('int64')

In [ ]:
county_metrics = county_metrics[['county_name', 'course_count', 'male_pop', 'female_pop', 'pop_over_65',
                                 'total_pop', 'pct_over_65', 'median_income', 'land_area_sqmi',
                                 'area_rank']]

In [ ]:
county_metrics['median_income'] = county_metrics['median_income'].astype('int64')

In [ ]:
county_metrics['land_area_sqmi'] = county_metrics['land_area_sqmi'].astype('float64')

In [ ]:
metrics_corr = county_metrics.drop(columns = {'county_name', 'male_pop', 'female_pop'}).corr()

In [ ]:
# Make rankings to look for anything standing out

In [ ]:
pop_ranks = (county_metrics[['county_name', 'course_count', 'total_pop']]
             .sort_values('total_pop', ascending = False).reset_index(drop = True).reset_index())

In [ ]:
pop_ranks = pop_ranks.rename(columns = {'index' : 'pop_rank'})

In [ ]:
pop_ranks['pop_rank'] = pop_ranks['pop_rank'] + 1

In [ ]:
pop_ranks = pop_ranks.sort_values('course_count', ascending = False)

In [ ]:
pop_ranks = pop_ranks.reset_index(drop = True)

In [ ]:
pct_65_plus_ranks = (county_metrics[['county_name', 'pct_over_65']]
             .sort_values('pct_over_65', ascending = False).reset_index(drop = True).reset_index())

In [ ]:
pct_65_plus_ranks = pct_65_plus_ranks.rename(columns = {'index' : 'pct_over_65_rank'})

In [ ]:
pct_65_plus_ranks['pct_over_65_rank'] = pct_65_plus_ranks['pct_over_65_rank'] + 1

In [ ]:
land_area_ranks = county_metrics[['county_name', 'land_area_sqmi', 'area_rank']]

In [ ]:
#merge all rankings for all ranks in one frame

In [ ]:
all_ranks = pop_ranks.merge(pct_65_plus_ranks, how = 'inner', on = 'county_name')

In [ ]:
all_ranks = all_ranks.merge(land_area_ranks, how = 'inner', on = 'county_name')

In [ ]:
all_ranks = all_ranks[['county_name', 'course_count', 'total_pop', 'pop_rank', 'pct_over_65',
                       'pct_over_65_rank', 'land_area_sqmi', 'area_rank']]

In [ ]:
all_ranks = all_ranks.sort_values('course_count', ascending = False)

In [ ]:
all_ranks_corr = all_ranks.drop(columns = {'county_name'}).corr()

In [ ]:
tn_center = (35.75, -86)

In [ ]:
tn_map = folium.Map(location = tn_center, zoom_start = 7)
folium.GeoJson(counties).add_to(tn_map)
for row, index in tn_courses.iterrows():
    loc = [index['lon'], index['lat']]
    pop = index['course_type'], index['holes']
    icon = folium.Icon(color = 'green', shadow_size = (0,0), prefix = 'fa')
    
    marker = folium.Marker(
        location = loc,
        popup = pop, icon = icon,
        tooltip = index['course_name'])
    
    marker.add_to(tn_map)

tn_map

In [ ]:
west_center = (35.8, -89.0)

In [ ]:
west_map = folium.Map(location = west_center, zoom_start = 8.25)
folium.GeoJson(west_counties_geo).add_to(west_map)
for row, index in west_courses.iterrows():
    loc = [index['lon'], index['lat']]
    pop = index['course_type'], index['holes']
    icon = folium.Icon(color = 'green', shadow_size = (0,0), prefix = 'fa')
    
    marker = folium.Marker(
        location = loc,
        popup = pop, icon = icon,
        tooltip = index['course_name'])
    
    marker.add_to(west_map)

west_map

In [ ]:
mid_center = (35.9, -86.5)

In [ ]:
mid_map = folium.Map(location = mid_center, zoom_start = 8.25)
folium.GeoJson(middle_counties_geo).add_to(mid_map)
for row, index in mid_courses.iterrows():
    loc = [index['lon'], index['lat']]
    pop = index['course_type'], index['holes']
    icon = folium.Icon(color = 'green', shadow_size = (0,0), prefix = 'fa')
    
    marker = folium.Marker(
        location = loc,
        popup = pop, icon = icon,
        tooltip = index['course_name'])
    
    marker.add_to(mid_map)

mid_map

In [ ]:
east_center = (35.75, -83.75)

In [ ]:
east_map = folium.Map(location = east_center, zoom_start = 8.25)
folium.GeoJson(east_counties_geo).add_to(east_map)
for row, index in east_courses.iterrows():
    loc = [index['lon'], index['lat']]
    pop = index['course_type'], index['holes']
    icon = folium.Icon(color = 'green', shadow_size = (0,0), prefix = 'fa')
    
    marker = folium.Marker(
        location = loc,
        popup = pop, icon = icon,
        tooltip = index['course_name'])
    
    marker.add_to(east_map)

east_map

In [ ]:
ax = counties.plot(figsize = (14,14))

tn_courses.plot(ax = ax, color = 'black')

In [ ]:
ax = west_counties_geo.plot(figsize = (12,12))

west_courses.plot(ax = ax, color = 'black')

In [ ]:
ax = middle_counties_geo.plot(figsize = (12,12))

mid_courses.plot(ax = ax, color = 'black')

In [ ]:
ax = east_counties_geo.plot(figsize = (12,12))

east_courses.plot(ax = ax, color = 'black')

In [ ]:
county_metrics.drop(columns = {'male_pop', 'female_pop', 'county_name'}).corr()

In [ ]:
top19 = county_metrics.sort_values('course_count', ascending = False).head(19)

In [ ]:
top19.to_csv('data/visualization_data/top19_counties.csv')

In [ ]:
# Make a query call to get course info for the top 19 counties

In [ ]:
top19_names = top19['county_name']

In [ ]:
top19_course_info = all_course_info.query('county_name in @ top19_names')